<a href="https://colab.research.google.com/github/PuppyQ08/CUDA-in-notebook/blob/main/vecadd_nvcc4jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi I am QQY. In this session, we are going to write CUDA codes related to the vector adding. It is pretty straightforward and we can get to learn how CUDA manipulate data on GPU and how CUDA Nsight compute provide you helpful insight.
We would use [nvcc4jupyter](https://github.com/andreinechaev/nvcc4jupyter) tool to run CUDA code in colab environment.

In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
#!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
#%load_ext cpp_plugh

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp99ksz3v6".


This is the util file and you can add any helper function you need here.

In [2]:
%%cuda_group_save --group shared --name "utils.h"
#include <math.h>
// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

This is the main body of files.

In [3]:
%%cuda_group_save --name "vector_add.cu" --group "vector_add"
// vector add kernel: C = A + B
__global__ void vadd(const float *A, const float *B, float *C, int ds){
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < ds) {
        C[idx] = A[idx] + B[idx];
    }
}

In [4]:
%%cuda_group_save --name "vector_add.h" --group "vector_add"
__global__ void vadd(const float *A, const float *B, float *C, int ds);

In [12]:
%%cuda_group_save --name "main.cu" --group "vector_add"
#include <stdio.h>
#include "utils.h"
#include "vector_add.h"

// Increase DSIZE to make the workload more substantial for profiling
const int DSIZE = 256 * 1024; // Increased from 4096
const int block_size = 256;

int main(){
    float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;

    // allocate space for vectors in host memory
    h_A = new float[DSIZE];
    h_B = new float[DSIZE];
    h_C = new float[DSIZE];

    // initialize vectors in host memory to random values (except for the
    // result vector whose values do not matter as they will be overwritten)
    for (int i = 0; i < DSIZE; i++) {
        h_A[i] = rand()/(float)RAND_MAX;
        h_B[i] = rand()/(float)RAND_MAX;
    }

    // allocate space for vectors in device memory
    cudaMalloc(&d_A, DSIZE*sizeof(float));
    cudaMalloc(&d_B, DSIZE*sizeof(float));
    cudaMalloc(&d_C, DSIZE*sizeof(float));
    cudaCheckErrors("cudaMalloc failure"); // error checking

    // copy vectors A and B from host to device:
    cudaMemcpy(d_A, h_A, DSIZE*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, DSIZE*sizeof(float), cudaMemcpyHostToDevice);
    cudaCheckErrors("cudaMemcpy H2D failure");

    // launch the vector adding kernel
    vadd<<<(DSIZE+block_size-1)/block_size, block_size>>>(d_A, d_B, d_C, DSIZE);
    cudaCheckErrors("kernel launch failure");

    // wait for the kernel to finish execution
    cudaDeviceSynchronize();
    cudaCheckErrors("kernel execution failure");

    cudaMemcpy(h_C, d_C, DSIZE*sizeof(float), cudaMemcpyDeviceToHost);
    cudaCheckErrors("cudaMemcpy D2H failure");

    printf("A[0] = %f\n", h_A[0]);
    printf("B[0] = %f\n", h_B[0]);
    printf("C[0] = %f\n", h_C[0]);
    return 0;
}

Here is the section for ncu, you can use it the get part of profiling results

In [6]:
!ncu --list-sets

---------- --------------------------------------------------------------------------- ------- -----------------
Identifier Sections                                                                    Enabled Estimated Metrics
---------- --------------------------------------------------------------------------- ------- -----------------
basic      LaunchStats, Occupancy, SpeedOfLight, WorkloadDistribution                  yes     144              
detailed   ComputeWorkloadAnalysis, LaunchStats, MemoryWorkloadAnalysis, MemoryWorkloa no      459              
           dAnalysis_Chart, Occupancy, SourceCounters, SpeedOfLight, SpeedOfLight_Roof                          
           lineChart, WorkloadDistribution                                                                      
full       ComputeWorkloadAnalysis, InstructionStats, LaunchStats, MemoryWorkloadAnaly no      613              
           sis, MemoryWorkloadAnalysis_Chart, MemoryWorkloadAnalysis_Tables, NumaAffin          

In [13]:
%cuda_group_run --group "vector_add" --profile --profiler-args "--section SpeedOfLight_RooflineChart" -c "--gpu-architecture sm_75"

==PROF== Connected to process 8192 (/tmp/tmp99ksz3v6/vector_add/cuda_exec.out)
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 4 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 8192
[8192] cuda_exec.out@127.0.0.1
  vadd(const float *, const float *, float *, int) (1024, 1, 1)x(256, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Roofline Chart
    INF   The ratio of peak float (fp32) to double (fp64) performance on this device is 32:1. The kernel achieved       
          close to 1% of this device's fp32 peak performance and 0% of its fp64 peak performance. See the Kernel        
          Profiling Guide (https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#roofline) for more details  
          on roofline analysis.                                                                                         

==WARNING== No metrics to show for section(s): SpeedOfLight_RooflineChart. Use --print-details 

In [ ]:
%cuda_group_run --group "vector_add"  -c "--gpu-architecture sm_75"

A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571



In [ ]:
%cuda_group_run --group "vector_add" --profile --profiler-args "--section SpeedOfLight" -c "--gpu-architecture sm_75"


==PROF== Connected to process 5525 (/tmp/tmppxfn1k8j/vector_add/cuda_exec.out)
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 8 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 5525
[5525] cuda_exec.out@127.0.0.1
  vadd(const float *, const float *, float *, int) (16, 1, 1)x(256, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.95
    SM Frequency                    Mhz       578.84
    Elapsed Cycles                cycle        4,372
    Memory Throughput                 %         1.68
    DRAM Throughput                   %         1.68
    Duration                         us         7.55
    L1/TEX Cache Throughput           %         6.23
    L2 Cache Throughput               %         1.19
   

In [17]:
%cuda_group_run --group "vector_add" -p -a "--section SpeedOfLight_HierarchicalSingleRooflineChart --print-details body" -c "--gpu-architecture sm_75"

==PROF== Connected to process 14790 (/tmp/tmp99ksz3v6/vector_add/cuda_exec.out)
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 4 passes
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 14790
[14790] cuda_exec.out@127.0.0.1
  vadd(const float *, const float *, float *, int) (1024, 1, 1)x(256, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Hierarchical Roofline Chart (Single Precision)
    Roofline Single Precision
    Table Name : Floating Point Operations Roofline (Single Precision)
    DRAM Roofline
    Peak Work
    ----------------------------------------- ----------- ------------
    Metric Name                               Metric Unit Metric Value
    ----------------------------------------- ----------- ------------
    Theoretical Predicated-On FFMA Operations        inst        5,120
    SM Frequency                                      Mhz       581.60
    ----------------------------------------- -----------

In [ ]:
%cuda_group_run --group "vector_add" -p -a "--section SchedulerStats" -c "--gpu-architecture sm_75

==PROF== Connected to process 9629 (/tmp/tmppxfn1k8j/vector_add/cuda_exec.out)
==PROF== Profiling "vadd" - 0: 0%....50%....100% - 1 pass
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
==PROF== Disconnected from process 9629
[9629] cuda_exec.out@127.0.0.1
  vadd(const float *, const float *, float *, int) (16, 1, 1)x(256, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: Scheduler Statistics
    ---------------------------- ----------- ------------
    Metric Name                  Metric Unit Metric Value
    ---------------------------- ----------- ------------
    One or More Eligible                   %         2.43
    Issued Warp Per Scheduler                        0.02
    No Eligible                            %        97.57
    Active Warps Per Scheduler          warp         1.99
    Eligible Warps Per Scheduler        warp         0.02
    ---------------------------- ----------- ------------

    OPT   Est. Local Speedup: 97.57%                                      